# Step 1: Data and Model Preparation


In [1]:
exp_name = 'exp_064'

In [2]:
%load_ext autoreload
%autoreload 2
import sys
import os
codebase = '../../'
sys.path.append(codebase)

In [3]:
from src.dataflow.vg_graph_loader_3 import VGGraphLoader
from src.dataflow.graph_batcher_4_rpn import GraphBatcher

In [4]:
from src.tensorflow.models.vsp_model_002 import Model
from src.tensorflow.engine.trainer_3 import Trainer
from src.tensorflow.engine.util import deploy, load_params
from src.util.util import pickle_save, pickle_load
from src.eval.sgg_evaluator_04 import Evaluator


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(1234)
np.random.seed(1235)

In [6]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_2.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)


In [7]:
img_ids, train_idx, test_idx = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'split_stanford.pkl'))
misc = pickle_load(os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'misc_stanford_2.pkl'))
img_no_ent = misc['img_no_ent']
img_no_pred = misc['img_no_pred']
image_size = misc['img_size']

In [8]:
train_idx = np.asarray(list(set(train_idx) - set(img_no_ent)))
np.random.shuffle(train_idx)
val_idx = train_idx[:1000]
train_idx = train_idx[1000:]

In [9]:
proposals = pickle_load(os.path.join(codebase, 'data', 'VG', 'xfeat_gtbox', 'iresnet_oi_lowprop', 'coords.pkl'))

In [10]:
batcher = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_gtbox', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1236, 
)
batcher.set_subset(train_idx)
batcher.shuffle()

In [11]:
batcher_val = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_gtbox', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_val.set_subset(val_idx)

In [12]:
tf.reset_default_graph()

In [13]:
model = Model(   
    dim_att_head_role_ent=[1024, 1024], 
    dim_att_head_role_pred=[1024, 1024],
    role_edge_mode='full_soft',
    stop_grad_role_edge=True,
    
    dim_state_ent=1024,
    dim_state_pred=1024,
    dim_emb_head_ent=[300],
    dim_conf_head_ent=[],
    dim_conf_head_pred=[],
    dim_emb_head_pred=[300],
    dim_init_head_ent=[1024],

    dim_message_send_head_pred2ent=[1024, 1024],
    dim_message_pool_head_pred2ent=[1024, 1024],
    dim_message_receive_head_pred2ent=[1024, 1024],    
    dim_message_send_head_ent2pred=[1024, 1024],
    dim_message_pool_head_ent2pred=[1024, 1024],
    dim_message_receive_head_ent2pred=[1024, 1024],
    
    num_mp_iter=3,     
    num_proposals=None,

    heat_role=1.0,
    heat_emb_mil=1.0,
    null_att_logit=0.0,
        
    num_align_iter=3,
    alignment_order='best',
    default_act_fn_name='leaky_relu',
    optimizer_type_generator=tf.train.AdamOptimizer,
    
    max_num_pred=100,
    dim_init_head_pred=[],
    init_state_type_pred='trainable',
    
    dim_proposal_feat=1536,
    emb_dict_ent=gl.noun_emb_dict,
    emb_dict_pred=gl.pred_emb_dict,    
    #image_size=448,
    #num_channels=3,
    debugging=False
)



Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Instructions for updating:
Use tf.cast instead.


In [14]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True)))
#sess = tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5)))
sess = tf.Session()
sess.run(tf.no_op())

# Step 2: Training


In [14]:
train_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': None,
    'filter_nonoverlap': True,
    'rank_triplet_confidence': False,
})
val_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': None,
    'filter_nonoverlap': True,
    'rank_triplet_confidence': False,
})


In [15]:
trainer = Trainer(
    model, sess, batcher, batcher_val, train_evaluator, val_evaluator,
    logdir=os.path.join(codebase, 'log', 'tfsummary', exp_name), 
    modeldir=os.path.join(codebase, 'checkpoint', exp_name),
)

In [16]:
trainer.train (
    max_steps=20000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=10000, 
    hyperparams = {
        'learning_rate_generator': 1e-5 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 10.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


../../src/eval/eval_utils.py:55: RuntimeWarning: invalid value encountered in true_divide
  iou = intsec / (area1[:, np.newaxis] + area2[np.newaxis, :] - intsec)
../../src/eval/graph_util_new_02.py:118: RuntimeWarning: invalid value encountered in greater
  overlap = (pw_iou(ent_box[i], ent_box[i]) > 0.0).astype(np.float32)
../../src/eval/graph_eval_3.py:219: RuntimeWarning: invalid value encountered in true_divide
  per_image_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))
../../src/eval/graph_eval_3.py:300: RuntimeWarning: invalid value encountered in true_divide
  per_image_recall.append(np.divide(len(gt_matched), len(gt_graph['pred_lbl'][i])))


Step 1000: loss = 3.046999931335449 (1510.052 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.015, 'PerImageOneHopMR_at100_iou_0.5': 0.22, 'PerImageOneHopMP_at50_iou_0.5': 0.026, 'PerImageOneHopMR_at50_iou_0.5': 0.2, 'PerImageLocPredMP_at100_iou_0.5': 0.032, 'PerImageLocPredMR_at100_iou_0.5': 0.478, 'PerImageLocPredMP_at50_iou_0.5': 0.054, 'PerImageLocPredMR_at50_iou_0.5': 0.428}
Validation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.014, 'PerImageOneHopMR_at100_iou_0.5': 0.216, 'PerImageOneHopMP_at50_iou_0.5': 0.025, 'PerImageOneHopMR_at50_iou_0.5': 0.206, 'PerImageLocPredMP_at100_iou_0.5': 0.03, 'PerImageLocPredMR_at100_iou_0.5': 0.481, 'PerImageLocPredMP_at50_iou_0.5': 0.054, 'PerImageLocPredMR_at50_iou_0.5': 0.445}
Step 2000: loss = 2.4860000610351562 (2769.792 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.017, 'PerImageOneHopMR_at100_iou_0.5': 0.247, 'PerImageOneHopMP_at50_iou_0.5': 0.029, 'PerImageOneHopMR_at50_iou_0.5

In [17]:
trainer.train (
    max_steps=10000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-6 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 10.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 21000: loss = 1.6770000457763672 (1179.816 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.029, 'PerImageOneHopMR_at100_iou_0.5': 0.404, 'PerImageOneHopMP_at50_iou_0.5': 0.05, 'PerImageOneHopMR_at50_iou_0.5': 0.363, 'PerImageLocPredMP_at100_iou_0.5': 0.05, 'PerImageLocPredMR_at100_iou_0.5': 0.7, 'PerImageLocPredMP_at50_iou_0.5': 0.085, 'PerImageLocPredMR_at50_iou_0.5': 0.626}
Validation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.024, 'PerImageOneHopMR_at100_iou_0.5': 0.32, 'PerImageOneHopMP_at50_iou_0.5': 0.043, 'PerImageOneHopMR_at50_iou_0.5': 0.294, 'PerImageLocPredMP_at100_iou_0.5': 0.047, 'PerImageLocPredMR_at100_iou_0.5': 0.631, 'PerImageLocPredMP_at50_iou_0.5': 0.081, 'PerImageLocPredMR_at50_iou_0.5': 0.576}
Step 22000: loss = 1.281999945640564 (2358.684 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.032, 'PerImageOneHopMR_at100_iou_0.5': 0.434, 'PerImageOneHopMP_at50_iou_0.5': 0.056, 'PerImageOneHopMR_at50_iou_0.

In [18]:
trainer.train (
    max_steps=5000,
    train_eval_rate=100, 
    validation_rate=100, 
    verbose_rate=1000, 
    summary_rate=100, 
    checkpoint_rate=1000, 
    hyperparams = {
        'learning_rate_generator': 1e-7 * batcher.batch_size,

        'loss_factor_g_mil_ent': 1.0,
        'loss_factor_g_mil_ent_conf': 0.0,
        'loss_factor_g_mil_pred': 1.0,
        'loss_factor_g_mil_pred_conf': 0.0,
        'loss_factor_g_mil_role': 10.0,
        'loss_factor_g_mil_box': 10.0,
        'loss_factor_g_aux_emb_ent': 0.0,
        'loss_factor_g_aux_emb_pred': 0.0,

        'confidence_weight_on_role': 0.0,
        'grad_clipping': 5.0,
        'mil_iou_offset': 0.01,
        'dropout_rate_g': 0.0,
    },
    validation_hyperparams = {
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
)


Step 31000: loss = 1.4850000143051147 (1164.865 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.035, 'PerImageOneHopMR_at100_iou_0.5': 0.479, 'PerImageOneHopMP_at50_iou_0.5': 0.06, 'PerImageOneHopMR_at50_iou_0.5': 0.426, 'PerImageLocPredMP_at100_iou_0.5': 0.057, 'PerImageLocPredMR_at100_iou_0.5': 0.781, 'PerImageLocPredMP_at50_iou_0.5': 0.096, 'PerImageLocPredMR_at50_iou_0.5': 0.69}
Validation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.025, 'PerImageOneHopMR_at100_iou_0.5': 0.325, 'PerImageOneHopMP_at50_iou_0.5': 0.044, 'PerImageOneHopMR_at50_iou_0.5': 0.294, 'PerImageLocPredMP_at100_iou_0.5': 0.048, 'PerImageLocPredMR_at100_iou_0.5': 0.627, 'PerImageLocPredMP_at50_iou_0.5': 0.083, 'PerImageLocPredMR_at50_iou_0.5': 0.565}
Step 32000: loss = 1.11899995803833 (2328.772 sec)
Training Batch Evaluation Results: {'PerImageOneHopMP_at100_iou_0.5': 0.035, 'PerImageOneHopMR_at100_iou_0.5': 0.49, 'PerImageOneHopMP_at50_iou_0.5': 0.059, 'PerImageOneHopMR_at50_iou_0

In [18]:
trainer.save_model('final_1')

# Step 3: Evaluation

In [15]:
load_params(sess, os.path.join(codebase, 'checkpoint', exp_name, 'ckpt-25000'))

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../../checkpoint/exp_064/ckpt-25000


In [16]:
gl = VGGraphLoader(
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'sg_stanford_with_duplicates.pkl'),
    os.path.join(codebase, 'metadata', 'VG', 'prep', 'stanford', 'word_emb_stanford_2.pkl'),
)


In [17]:
test_evaluator = Evaluator({        
    'graph_loader': gl,
    'proposals': proposals,
    'num_proposals': None,
    'filter_nonoverlap': True,
    'rank_triplet_confidence': False,
})


In [18]:
test_idx = np.asarray(list(set(test_idx) - set(img_no_ent)))

In [19]:
batcher_test = GraphBatcher(
    gl, 32, proposals,
    os.path.join(codebase, 'data', 'VG', 'xfeat_gtbox', 'iresnet_oi_lowprop', 'feats_float32_20x1536.lmdb'), 
    dim_feats=1536,
    seed=1237, 
)
batcher_test.set_subset(test_idx)

In [20]:
#test_evaluator.reset()
deploy(sess, model, batcher_test, test_evaluator, model.eval_ops,
    hyperparams={
        'dropout_rate_g': 0.0,
        'confidence_weight_on_role': 0.0
    },
    ignore_errors=False,
    debug_mode=False,
    verbose_rate=100
)


Deploying
Batch 1/997 done in 1 seconds. Spent time for read=0.01 (avg 0.01), processing=1.37 (avg 1.37), write=0.00 (avg 0.00) seconds.
Batch 101/997 done in 8 seconds. Spent time for read=0.00 (avg 0.01), processing=0.06 (avg 0.08), write=0.00 (avg 0.00) seconds.
Batch 201/997 done in 16 seconds. Spent time for read=0.00 (avg 0.01), processing=0.07 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 301/997 done in 23 seconds. Spent time for read=0.00 (avg 0.01), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 401/997 done in 31 seconds. Spent time for read=0.00 (avg 0.01), processing=0.06 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 501/997 done in 38 seconds. Spent time for read=0.01 (avg 0.01), processing=0.08 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 601/997 done in 46 seconds. Spent time for read=0.01 (avg 0.01), processing=0.07 (avg 0.07), write=0.00 (avg 0.00) seconds.
Batch 701/997 done in 54 seconds. Spent time for read=0.01 (avg 0.01), processing=0.

In [26]:
test_results = test_evaluator.evaluate()
test_results

{'PerImageOneHopMP_at100_iou_0.5': 0.025612059229514372,
 'PerImageOneHopMR_at100_iou_0.5': 0.34125216158398747,
 'PerImageOneHopMP_at50_iou_0.5': 0.0444924080813151,
 'PerImageOneHopMR_at50_iou_0.5': 0.315035548678333,
 'PerImageLocPredMP_at100_iou_0.5': 0.04834201279959844,
 'PerImageLocPredMR_at100_iou_0.5': 0.646035299673522,
 'PerImageLocPredMP_at50_iou_0.5': 0.08310954950432926,
 'PerImageLocPredMR_at50_iou_0.5': 0.5913678575900829}